In [1]:
import numpy as np
import scipy.sparse as sp
import relpy as rp
import matplotlib.pyplot as plt
import pydotplus
from IPython.display import Image
import time

In [2]:
def draw(x):
    dotf = x.todot()
    g = pydotplus.graph_from_dot_data(dotf)
    return Image(g.create_png())

In [3]:
with open('sip.py', 'w') as f:
    script = rp.parse('./avail.txt')
    f.write(script)

include ./param_sharpe
include ./models.txt


In [4]:
from sip import *

In [5]:
_time = rp.Parameter('t')
gamma = bind_gamma(gamma_die, gamma_hang)
pxy_gamma = bind_pxy_gamma(pxy_gamma_die, pxy_gamma_hang)
pairup = ctmc_pairup(b, beta_m, c, d, delta1, delta2, delta_m, e, gamma, mu, phi, q, r, rou_a, rou_m)
pxy_pairup = ctmc_pxy_pairup(pxy_b, pxy_beta_m, pxy_c, pxy_d, pxy_delta1, pxy_delta2, pxy_delta_m, pxy_e, pxy_gamma, pxy_mu, pxy_phi, pxy_q, pxy_r, pxy_rou_a, pxy_rou_m)
appserver = ctmc_appserver(b, beta_m, d, delta1, delta2, delta_m, e, gamma, mu, q, r, rou_a, rou_m)
proxy = ctmc_proxy(pxy_b, pxy_beta_m, pxy_d, pxy_delta1, pxy_delta2, pxy_delta_m, pxy_e, pxy_gamma, pxy_mu, pxy_q, pxy_r, pxy_rou_a, pxy_rou_m)
midplane = ctmc_midplane(alpha_sp, c_mp, mu_mp, plambda_mp)
cooling = ctmc_cooling(alpha_sp, mu_2c, mu_c, plambda_c)
power = ctmc_power(alpha_sp, c_ps, mu_2ps, mu_ps, plambda_ps)
processor = ctmc_processor(alpha_sp, mu_cpu, plambda_cpu)
memory = ctmc_memory(alpha_sp, mu_mem, plambda_mem)
disk = ctmc_disk(alpha_sp, chi_hd, mu_2hd, mu_hd, plambda_hd)
base = ctmc_base(alpha_sp, mu_base, plambda_base)
OS = ctmc_OS(alpha_sp, b_os, beta_os, delta_os, mu_os, plambda_os)
nic = ctmc_nic(alpha_sp, mu_nic, plambda_nic)
switch = ctmc_switch(alpha_sp, mu_esw, plambda_esw)
f_esw = bind_f_esw(plambda_esw, switch)
f_nic = bind_f_nic(nic, plambda_nic)
u_esw = bind_u_esw(f_esw, switch)
u_nic = bind_u_nic(f_nic, nic)
network = ftree_network(_time, plambda_esw, plambda_nic)
CM = ftree_CM(cooling, midplane, power)
BLADE = ftree_BLADE(OS, base, disk, memory, processor)
CLUSTER = ftree_CLUSTER(BLADE, CM, appserver, nic, proxy, switch)


In [6]:
start = time.time()
x = CLUSTER.seval(env)
print("elapsed_time:{0} [sec]".format(time.time() - start))

elapsed_time:0.1439347267150879 [sec]


In [7]:
params = CLUSTER.paramlist()
print(len(params))

58


In [8]:
start = time.time()
ddx = CLUSTER.sderiv2(env, params[0], params[1])
print("elapsed_time:{0} [sec]".format(time.time() - start))
ddx

elapsed_time:1.427675724029541 [sec]


4.217352743086264e-14

In [9]:
start = time.time()
dx = [CLUSTER.sderiv(env, x) for x in params]
print("elapsed_time:{0} [sec]".format(time.time() - start))

elapsed_time:24.015148878097534 [sec]


In [10]:
start = time.time()
dxx = [[CLUSTER.sderiv2(env, x1, x2) for x1 in params] for x2 in params]
print("elapsed_time:{0} [sec]".format(time.time() - start))

elapsed_time:5491.317338943481 [sec]


In [12]:
dxx=np.array(dxx)

In [13]:
print(dxx)

[[3.74553994e-06 4.21735274e-14 4.53546392e-07 ... 8.11547531e-14
  1.21992847e-15 1.74748862e-15]
 [4.21735274e-14 2.27750999e-15 1.46183496e-13 ... 5.31125201e-13
  6.58801729e-17 9.43701660e-17]
 [4.53546392e-07 1.46183496e-13 2.36003725e-07 ... 2.81301714e-13
  4.22856280e-15 6.05721199e-15]
 ...
 [8.11547531e-14 5.31125201e-13 2.81301714e-13 ... 8.43350986e-15
  1.26773583e-16 1.81597035e-16]
 [1.21992847e-15 6.58801729e-17 4.22856280e-15 ... 1.26773583e-16
  3.90331663e-17 2.72978950e-18]
 [1.74748862e-15 9.43701660e-17 6.05721199e-15 ... 1.81597035e-16
  2.72978950e-18 2.02599859e-16]]


In [14]:
dx=np.array(dx)

In [15]:
dx

array([-7.43218296e-07, -2.63371288e-14, -1.85015233e-07, -2.25008826e-16,
       -1.71310400e-09,  8.04791253e-06, -3.04102242e-05,  4.66499622e-18,
       -3.37655498e-14, -1.53325349e-20, -4.90424853e-14, -8.44138744e-16,
       -1.54179360e-08, -1.42405025e-13,  4.45075951e-10,  4.45075951e-10,
       -5.26742576e-15, -1.72405029e-13, -1.41719354e-15, -3.35856203e-08,
       -4.71827074e-10, -2.46291439e-14, -2.18954005e-10, -4.99287294e-13,
       -3.29303906e-10, -6.07515184e-10, -8.04788488e-07, -5.48772578e-16,
       -7.70896802e-10, -3.13966854e-12,  1.62086755e-03,  1.29116499e-05,
        3.24175330e-03,  1.46895495e-05,  1.07840626e-03,  6.48345065e-03,
        1.01802646e+00,  1.02281836e-08,  4.41199658e-04,  1.23485666e-01,
       -7.20300675e-09, -6.20691185e-11, -9.23462404e-09, -4.14573250e-18,
       -1.34128910e-08, -2.30865601e-10, -3.91533612e-08,  1.15376191e-04,
        1.15376191e-04, -1.44060135e-09, -3.47674836e-08, -1.38894429e-08,
       -6.04686274e-11, -

In [16]:
len(dxx)

58

In [18]:
np.shape(dxx)

(58, 58)

In [20]:
dxx[1,0]

4.217352743086264e-14

In [21]:
dxx[0,1]

4.217352743086264e-14

In [22]:
np.save('dxx.npy', dxx)